In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
sgm = lambda x: 1/(1+np.exp(-x))
nrand = np.random.random
zerov = np.zeros

In [ ]:
class RBM:
    '''A restricted Boltzmann machine model.'''
    def __init__(self, n_variables, n_hidden, spin = False):
        sigma = np.sqrt(4./(n_hidden+n_variables))
        self.a = sigma*(2*nrand(n_variables)-1)
        self.b = zerov(n_hidden)
        self.w = sigma*(2*nrand((n_variables,n_hidden))-1)
        self.min = -1*spin # Evaluates to zero if the model is not spin-based
        self.gap = 1 + spin # Evaluates to two if the model is spin-based

    def activate(self, v, w, b, beta):
        '''Turns the neurons of a forward layer on, according to a sigmoid probability function.'''
        vec = v@w + b # The neurons evaluation
        l = len(vec) # sample size
        prob = sgm(beta*vec)
        out = np.full(prob.shape,self.min)
        out[nrand(prob.shape) < prob] = 1
        return out
    
    def sample(self, v_data, steps = 1):
        h_data = self.activate(v_data, self.w, self.b, self.gap)
        h_model = h_data
        for _ in range(steps):
            v_model = self.activate(h_model, self.w.T, self.a, self.gap)
            h_model = self.activate(v_model, self.w, self.b, self.gap)
        return h_data, v_model, h_model
    
    def grad(self, v_data, h_data, v_model, h_model):
        da = np.average(v_data - v_model, axis = 0)
        db = np.average(h_data - h_model, axis = 0)
        dw = sum(np.outer(v_data[i].T,h_data[i])-np.outer(v_model[i].T,h_model[i]) for i in range(len(v_data)))/len(v_data)
        #np.average(np.outer(v_data,h_data)-np.outer(v_model,h_model), axis = 1)
        return da, db, dw
    
    '''PLOTTING PURPOSES'''
    # This function set the coordinates (dots) for the unit of the visible layers and
    # the one in the hidden layer in the plot
    def create_coord(self,np,x0):
        x = [x0] * np
        y = list(range(np)) # weird way to spell np.arange(0,10)
    
        for i in range(np):
            y[i] = y[i]/(np-1.) - 0.5
        return (x,y)
    
    '''PLOTTING PURPOSES'''
    def mycolor(self,val):
        if val > 0:
            return "red"
        elif val < 0:
            return "blue"
        return "black"
    
    '''PLOTTING PURPOSES'''
    # This makes actually the plot, it display all the units and all the weight with a
    # color and an intensity (the thickness)
    def plotgraph(self,epoch=0,):
        (x1,y1) = self.create_coord(len(self.a),0)
        (x2,y2) = self.create_coord(len(self.b),1)
            
        A = 2./self.w.max()
        for i in range(len(self.a)): # L are visible (LEFT)
            for j in range(len(self.b)): # M are hidden (RIGHT)
                ex, ey, col = (x1[i],x2[j]), (y1[i],y2[j]), self.mycolor(self.w[i][j])
                plt.plot(ex,ey,col,zorder=1, lw=A*np.abs(self.w[i][j]))
        A = 300. /(self.a.max()+self.b.max())
    
        # Left dots
        for i in range(len(self.a)):
            plt.scatter(x1[i],y1[i],s=A*np.abs(self.a[i]),zorder=2,c=self.mycolor(self.a[i]))
        
        # Right dots
        for j in range(len(self.b)):
            plt.scatter(x2[j],y2[j],s=A*np.abs(self.b[j]),zorder=2,c=self.mycolor(self.b[j]))
        
        plt.title(f">0 red, <0 blue, epoch={epoch}")
        plt.show()

    def train(self, data, n_epochs = 10, batch_size = 30, learning_rate = 1, nplot = 0):
        batches = len(data)//batch_size
        n_variables = len(self.a)
        n_hidden = len(self.b) 
        
        for epoch in range(n_epochs):
            np.random.shuffle(data)
            for i in range(batches):
                v = data[i*batch_size:(i+1)*batch_size]
                h, v_m, h_m = self.sample(v)
                ca, cb, cw = self.grad(v,h,v_m,h_m)
                self.a += learning_rate*ca
                self.b += learning_rate*cb
                self.w += learning_rate*cw
            learning_rate = learning_rate/(0.05*learning_rate+1)
            # I just want to plot nplot times (NOT TOTALLY CORRECT, just a detail though)
            if nplot:
                if (epoch)%(int(n_epochs/nplot)) == 0:
                    self.plotgraph(epoch)
                    print("l_rate=",learning_rate)
        if nplot:
            self.plotgraph(epoch)
            print("l_rate=",learning_rate)


In [ ]:
data = np.loadtxt('DATA/dataRBM_q0.1.csv', delimiter = ',')
rbm = RBM(len(data[0]),3, spin = True)
rbm.train(data, n_epochs = 50, batch_size = 500, nplot = 3)

In [ ]:
np.random.shuffle(data)
_, v_fantasy, h_fantasy = rbm.sample(data,1)

In [ ]:
v_fantasy